# 12.5.2 Continuous monitoring and finite discrete state space

This notebook is dedicated to the Case 4 when $\mu_{13}$ is not constant 

The notebook is structured as follows:
+ Functions
+ Case 4 a: rate $\mu_{13}$ is constant (for comparison)
+ Case 4 b: rate $\mu_{13}$ is not constant: it depends on the time spend in state 2 according to a linear relation given in function `OneHistory`
+ Compare cases



In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

%matplotlib notebook
import matplotlib.pyplot as plt

### Functions

In [2]:
def RandExp(rate):
    '''Draw random samples from an exponential distribution.
    If rate is null, output is set to infinity.'''
    if rate==0:
        return np.inf
    else:
        return np.random.exponential(scale=1/rate)

In [3]:
def UpdateVar(horizonTime, currentTime, currentState, dT, nextState, 
              timeState, transMatrix):
    '''Update system state and time spent in the different states and achieved transitions.'''
    currentTime = currentTime+dT
    flagSim = True
    if currentTime>=horizonTime:
        flagSim = False
        dT = dT-(currentTime-horizonTime)
        currentTime = currentTime+dT
    timeState[currentState] = timeState[currentState]+dT
    transMatrix[currentState,nextState] = transMatrix[currentState,nextState]+1
    currentState = nextState
    return (flagSim, currentTime, currentState, timeState, transMatrix)

In [4]:
def OneHistory(para):
    '''Simulate one history of the system until simulation time horizon is reached.
    '''
    # Definition of the interpolation function to determine mu3 depending on the time spent in state 2
    fmu3 = interp1d(np.array([para['mu13']['tinf'], para['mu13']['tsup']]),
                    np.array([para['mu13']['vinf'], para['mu13']['vsup']]),
                    bounds_error=False, fill_value=(para['mu13']['vinf'], para['mu13']['vsup']))
    # Intial conditions
    currentState = 3
    currentTime = 0
    flagSim = True
    # Counters initialization
    nState = 4
    timeState = np.zeros(nState)
    transMatrix = np.zeros((nState, nState))
    while flagSim:
        if (currentState==3):
            # Evaluate concurrent transitions and determine the actual one
            dT = RandExp(rate=para['lambda32'])
            nextState = 2
        elif (currentState==2):
            # Evaluate concurrent transitions and determine the actual one
            dT23 = RandExp(rate=para['mu23'])
            dT21 = RandExp(rate=para['lambda21'])
            dT = np.min([dT23, dT21])
            if (dT==dT23):
                nextState = 3
            elif (dT==dT21):
                nextState = 1
            else:
                raise ValueError('Approximation error')
        elif (currentState==1):
            # POSSIBLY NOT CONSTANT TRANSITION RATE
            mu13 = fmu3(dT)
            # Evaluate concurrent transitions and determine the actual one
            dT12 = RandExp(rate=para['mu12'])
            dT13 = RandExp(rate=mu13)
            dT10 = RandExp(rate=para['lambda10'])
            dT = np.min([dT12, dT13, dT10])
            if (dT==dT12):
                nextState = 2
            elif (dT==dT13):
                nextState = 3
            elif (dT==dT10):
                nextState = 0
            else:
                raise ValueError('Approximation error')
        elif (currentState==0):
            # Evaluate concurrent transitions and determine the actual one
            dT = RandExp(rate=para['mu03'])
            nextState = 3
        else:
            raise ValueError('Unknown state')
        # Update variables
        (flagSim, currentTime, currentState, timeState, transMatrix) = UpdateVar(
            para['horizon'], currentTime, currentState, dT,  nextState, 
            timeState, transMatrix)
    return (timeState, transMatrix)

In [5]:
def NHistories(nbN, para):
    '''Simulate several histories and aggregate time spent in the different states and achieved
    transitions (Monté-carlo approach).
    '''
    # Simulate a first history to assess output shapes
    (timeStateOne, transMatrixOne) = OneHistory(para)
    timeState = np.empty((nbN, timeStateOne.shape[0]))
    transMatrix = np.empty((nbN, transMatrixOne.shape[0], transMatrixOne.shape[1]))
    # Main loop
    for id in range(nbN):
        (timeState[id,:], transMatrix[id,:,:]) = OneHistory(para)
    return (timeState, transMatrix)

In [6]:
def GetResults(timeState, transMatrix, case):
    '''Get and display simulations results'''
    # Probability of each states
    res = {'Case': case}
    for id in range(4):
        res['State{:d}'.format(id)] = np.mean(timeState[:, id]/np.sum(timeState[:, :], axis=1))
        print('State {:d} -> Probability: {:.5f}'.format(id, res['State{:d}'.format(id)], axis=1))
    # Average simulated time
    res['AveSimTime'] = np.mean(np.sum(timeState[:, :], axis=1))
    print('Average simulated time (h): {:.2f}'.format(res['AveSimTime']))
    # Average operating time
    res['AveOpeTime'] = np.mean(np.sum(timeState[:, 1:], axis=1))
    print('Average operating time (h): {:.2f}'.format(res['AveOpeTime']))
    # Average repairing time
    res['aveRepTime'] = np.mean(timeState[:, 0])
    print('Average repairing time (h): {:.2f}'.format(res['aveRepTime']))
    # Average number of corrective maintenance
    res['aveCorMaint'] = np.mean(transMatrix[:, 0, 3])
    print('Average number of corrective maintenance: {:.2f}'.format(res['aveCorMaint']))
    # Average number of preventive maintenance
    res['avePreMaint'] = np.mean(transMatrix[:, 1, 2])+np.mean(transMatrix[:, 1, 3]+np.mean(transMatrix[:, 2, 3])) 
    print('Average number of preventive maintenance: {:.2f}'.format(res['avePreMaint']))    
    return res

### CASE 4 : TWO LEVELS PREVENTIVE MAINTENANCE (a. $\mu_{13}$ constant)

In [7]:
# Parameters
nbHist = int(1e5)
# By setting same values at `vinf` and `vsup`, a constant rate is forced
para = {
    'horizon' : 10*365*24,
    'lambda32': 3e-5,
    'lambda21': 3e-5,
    'lambda10': 3e-5,
    'mu03': 1e-3,
    'mu12': 0,
    'mu13': {'tinf': 100, 'vinf': 1e-3, 'tsup': 2500, 'vsup': 1e-3},
    'mu23': 1e-3}
# Simulation
(timeState, transMatrix) = NHistories(nbHist, para)
# Get simulations results
print('Case 4: Two levels preventive maintenance\n')
res4a = GetResults(timeState, transMatrix, 4)

Case 4: Two levels preventive maintenance

State 0 -> Probability: 0.00003
State 1 -> Probability: 0.00082
State 2 -> Probability: 0.02803
State 3 -> Probability: 0.97112
Average simulated time (h): 87600.00
Average operating time (h): 87597.60
Average repairing time (h): 2.40
Average number of corrective maintenance: 0.00
Average number of preventive maintenance: 2.55


In [8]:
### CASE 4 : TWO LEVELS PREVENTIVE MAINTENANCE (b. $\mu_{13}$ not constant)

In [9]:
# Parameters
nbHist = int(1e5)
para = {
    'horizon' : 10*365*24,
    'lambda32': 3e-5,
    'lambda21': 3e-5,
    'lambda10': 3e-5,
    'mu03': 1e-3,
    'mu12': 0,
    'mu13': {'tinf': 100, 'vinf': 1e-5, 'tsup': 2500, 'vsup': 1e-1},
    'mu23': 1e-3}
# Simulation
(timeState, transMatrix) = NHistories(nbHist, para)
# Get simulations results
print('Case 4: Two levels preventive maintenance\n')
res4b = GetResults(timeState, transMatrix, 4)

Case 4: Two levels preventive maintenance

State 0 -> Probability: 0.00005
State 1 -> Probability: 0.00151
State 2 -> Probability: 0.02796
State 3 -> Probability: 0.97048
Average simulated time (h): 87600.00
Average operating time (h): 87595.80
Average repairing time (h): 4.20
Average number of corrective maintenance: 0.00
Average number of preventive maintenance: 2.54


In [10]:
### Compare cases

In [11]:
df = pd.DataFrame([res4a, res4b]).set_index('Case')
df

,State0,State1,State2,State3,AveSimTime,AveOpeTime,aveRepTime,aveCorMaint,avePreMaint
Case,,,,,,,,,
4,0.000027,0.000819,0.028032,0.971122,87600.0,87597.600955,2.399045,0.00236,2.54559
4,0.000048,0.001515,0.027962,0.970475,87600.0,87595.802581,4.197419,0.00429,2.54249
